# Lesson - Machine Learning II: Evaluating Model Performance

We have developed a function that can predict the price for any living space we want to list as long as we know the number of people it can accommodate. The function we have written represents a **machine learning model**, which means that it outputs a prediction based on the input to the model.

A simple way to test the quality of our model is to:

- split the dataset into 2 partitions:

 - the training set: contains the majority of the rows (75%)
 - the test set: contains the remaining minority of the rows (25%)
- use the rows in the training set to predict the price value for the rows in the test set
 - add new column named predicted_price to the test set
- compare the predicted_price values with the actual price values in the test set to see how accurate the predicted values were.


This validation process, where we use the training set to make predictions and the test set to predict values for, is known as **train/test validation**. Whenever we're performing machine learning, we want to perform validation of some kind to ensure that our machine learning model can make good predictions on new data. While train/test validation isn't perfect, we'll use it to understand the validation process, to select an error metric.


We will modify the `predict_price` function to use only the rows in the training set, instead of the full dataset, to find the nearest neighbors, average the price values for those rows, and return the predicted price value. Then, we'll use this function to predict the price for just the rows in the test set. Once we have the predicted price values, we can compare with the true price values and start to understand the model's effectiveness

predict_price
```
def predict_price(new_listing):
    ## DataFrame.copy() performs a deep copy
    temp_df = dc_listings.copy()
    temp_df['distance'] = temp_df['accommodates'].apply(lambda x: np.abs(x - new_listing))
    temp_df = temp_df.sort_values('distance')
    nearest_neighbor_prices = temp_df.iloc[0:5]['price']
    predicted_price = nearest_neighbor_prices.mean()
    return(predicted_price)
```    

**Exercise**

- Within the `predict_price` function, change the Dataframe that `temp_df` is assigned to. Change it from `dc_listings` to train_df, so only the training set is used.
- Use the Series method apply to pass all of the values in the accommodates column from `test_df` through the `predict_price `function.
- Assign the resulting Series object to the `predicted_price` column in test_df.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# Setting pandas display options for large data
pd.options.display.max_rows = 200
pd.options.display.max_columns = 150

In [20]:
dc_listings = pd.read_csv("dc_airbnb.csv")
stripped_commas = dc_listings['price'].str.replace(',', '')
stripped_dollars = stripped_commas.str.replace('$', '')
dc_listings['price'] = stripped_dollars.astype('float')
train_df = dc_listings.iloc[0:2792]
test_df = dc_listings.iloc[2792:]



In [21]:
def predict_price(new_listing):
    ## DataFrame.copy() performs a deep copy
    temp_df = train_df.copy()
    temp_df['distance'] = temp_df['accommodates'].apply(lambda x: np.abs(x - new_listing))
    temp_df = temp_df.sort_values('distance')
    nearest_neighbor_prices = temp_df.iloc[0:5]['price']
    predicted_price = nearest_neighbor_prices.mean()
    return(predicted_price)


145.8

In [6]:
# Apply to test dataset

test_df['predicted_price'] = test_df['accommodates'].apply(predict_price)
print(test_df[['price','predicted_price']].head())

      price  predicted_price
2792  120.0            104.0
2793  140.0            177.4
2794  299.0            145.8
2795   85.0            177.4
2796  175.0            187.2


<ipython-input-6-f21e15fa2f42>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predicted_price'] = test_df['accommodates'].apply(predict_price)


### Error Metrics

We now need a metric that quantifies how good the predictions were on the test set. This class of metrics is called an error metric. As the name suggests, an error metric quantifies how inaccurate our predictions were from the actual values. In our case, the error metric tells us how off our predicted price values were from the actual price values for the living spaces in the test dataset.

We could start by calculating the difference between each predicted and actual value and then averaging these differences. This is referred to as mean error but isn't an effective error metric for most cases. Mean error treats a positive difference differently than a negative difference, but we're really interested in how far off the prediction is in either the positive or negative direction. If the true price was 200 dollars and the model predicted 210 or 190 it's off by 10 dollars either way.

We can instead use the **mean absolute error (MAE)**, where we compute the absolute value of each error before we average all the errors.
$$MAE = \frac{1}{n} \sum_{k=1}^{n} \lvert (actual_1 - predicted_1) \rvert + \cdots + \lvert (actual_n - predicted_n) \rvert$$

**Exercise**

- Use `numpy.absolute()` to calculate the mean absolute error between `predicted_price` and `price`.
Assign the MAE to `mae`

In [10]:
test_df['error'] = np.absolute(test_df.predicted_price - test_df.price)
mae = test_df.error.mean()
print(mae)

56.29001074113876


<ipython-input-10-958cdd88b570>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['error'] = np.absolute(test_df.predicted_price - test_df.price)


### Mean Squared Error
For many prediction tasks, we want to penalize predicted values that are further away from the actual value far more than those closer to the actual value.

We can instead take the mean of the squared error values, which is called the **mean squared error or MSE** for short. The MSE makes the gap between the predicted and actual values more clear. A prediction that's off by 100 dollars will have an error (of 10,000) that's 100 times more than a prediction that's off by only 10 dollars (which will have an error of 100).

$$MSE = \frac{1}{n} \sum_{k=1}^{n} (actual_1 - predicted_1)^{2} + \cdots + (actual_n - predicted_n)^{2}$$
where `n` represents the number of rows in the test set.


**Exercise**

Calculate the MSE value between the `predicted_price` and `price columns` and assign to `mse`.

In [11]:
test_df['squared_error'] = (test_df.predicted_price - test_df.price)**2
mse = test_df.squared_error.mean()
mse

<ipython-input-11-7502a23a5383>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['squared_error'] = (test_df.predicted_price - test_df.price)**2


18646.525370569325

### Training Another Model

The model we trained achieved a mean squared error of around 18646.5. Is this a high or a low mean squared error value? What does this tell us about the quality of the predictions and the model? By itself, the mean squared error value for a single model isn't all that useful.

The units of mean squared error in our case is dollars squared (not dollars), which makes it hard to reason about intuitively as well. We can, however, train another model and then compare the mean squared error values to see which model performs better on a relative basis. A low error metric means that the gap between the predicted list price and actual list price values is low while a high error metric means the gap is high.

We will now train another model, this time using the `bathrooms` column, and compare MSE values.



**Exercise**

- Modify the `predict_price` function to use the `bathrooms` column instead of the `accommodates` column to make predictions.
- Apply the function to `test_df` and assign the resulting Series object containing the predicted price values to the `predicted_price` column in `test_df`.
- Calculate the squared error between the price and predicted_price columns in test_df and assign the resulting Series object to the `squared_error` column in `test_df`.
- Calculate the mean of the squared_error column in `test_df` and assign to `mse`.
- Use the print function  to display the MSE value.

In [12]:
def predict_price(new_listing):
    ## DataFrame.copy() performs a deep copy
    temp_df = train_df.copy()
    temp_df['distance'] = temp_df['bathrooms'].apply(lambda x: np.abs(x - new_listing))
    temp_df = temp_df.sort_values('distance')
    nearest_neighbor_prices = temp_df.iloc[0:5]['price']
    predicted_price = nearest_neighbor_prices.mean()
    return(predicted_price)

test_df['predicted_price'] = test_df['bathrooms'].apply(predict_price)
test_df['squared_error'] = (test_df.predicted_price - test_df.price)**2
mse = test_df.squared_error.mean()
mse

<ipython-input-12-4da8e0f0bd8e>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predicted_price'] = test_df['bathrooms'].apply(predict_price)
<ipython-input-12-4da8e0f0bd8e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['squared_error'] = (test_df.predicted_price - test_df.price)**2


18405.444081632548

### Root Mean Squared Error (RMSE)

While comparing MSE values helps us identify which model performs better on a relative basis, it doesn't help us understand if the performance is good enough in general. This is because the units of the MSE metric are squared (in this case, dollars squared). An MSE value of 16377.5 dollars squared doesn't give us an intuitive sense of how far off the model's predictions are from the true price value in dollars.

**Root mean squared error or RMSE** is an error metric whose units are the base unit (in our case, dollars). This error metric is calculated by taking the square root of the MSE value:

$$RMSE = \sqrt{MSE}$$

Since the RMSE value uses the same units as the target column, we can understand how far off in real dollars we can expect the model to perform. 
We will now calculate the RMSE value of the model we trained using the bathrooms column.

In [13]:
def predict_price(new_listing):
    ## DataFrame.copy() performs a deep copy
    temp_df = train_df.copy()
    temp_df['distance'] = temp_df['bathrooms'].apply(lambda x: np.abs(x - new_listing))
    temp_df = temp_df.sort_values('distance')
    nearest_neighbor_prices = temp_df.iloc[0:5]['price']
    predicted_price = nearest_neighbor_prices.mean()
    return(predicted_price)

test_df['predicted_price'] = test_df['bathrooms'].apply(predict_price)f
test_df['squared_error'] = (test_df.predicted_price - test_df.price)**2
mse = test_df.squared_error.mean()
import math
rmse = math.sqrt(mse)
rmse

<ipython-input-13-ac89626193d1>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predicted_price'] = test_df['bathrooms'].apply(predict_price)
<ipython-input-13-ac89626193d1>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['squared_error'] = (test_df.predicted_price - test_df.price)**2


135.6666653295221

### Comparing MAE and RMSE

The model achieved an RMSE value of approximately 135.6, which implies that we should expect for the model to be off by 135.6 dollars on average for the predicted price values. Given that most of the living spaces are listed at just a few hundred dollars, we need to reduce this error as much as possible to improve the model's usefulness.

We discussed a few different error metrics we can use to understand a model's performance. As mentioned earlier, these individual error metrics are helpful for comparing models. To better understand a specific model, we can compare multiple error metrics for the same model. This requires a better understanding of the mathematical properties of the error metrics.

If we look at the equation for MAE:
$$MAE = \frac{1}{n} \sum_{k=1}^{n} \lvert (actual_1 - predicted_1) \rvert + \cdots + \lvert (actual_n - predicted_n) \rvert$$

notice that that the differences between predicted and actual values grow linearly. A prediction that's off by 10 dollars has a 10 times higher error than a prediction that's off by 1 dollar.

If we look at the equation for RMSE, however:
$$RMSE = \sqrt { \frac{ \sum_{k=1}^{n} (actual_1 - predicted_1)^2 + \cdots + (actual_n - predicted_n)^2 } {n} }$$

We can notice that each error is squared before the square root of the sum of all the errors is taken. This means that the individual errors grows quadratically and has a different effect on the final RMSE value.

We will look at an example using different data entirely. Below are 2 Series objects containing 2 sets of errors and assigned to errors_one and errors_two. 

- Calculate the MAE for `errors_one` and assign to `mae_one`.
- Calculate the RMSE for `errors_one` and assign to `rmse_one`.
- Calculate the MAE for `errors_two` and assign to `mae_two`.
- Calculate the RMSE for `errors_two` and assign to `rmse_two`

In [18]:
import math

def mae(error):
    mae = sum(error)/len(error)
    return mae

def rmse(error):
    squares = [] 
    for value in error:
        square = value**2
        squares.append(square)
    rmse = math.sqrt(sum(squares)/ len(error))
    return rmse
    
errors_one = pd.Series([5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10])
errors_two = pd.Series([5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 1000])

mae_one = mae(errors_one)
rmse_one = rmse(errors_one)
mae_two = mae(errors_two)
rmse_two = rmse(errors_two)
print(mae_one, rmse_one, mae_two, rmse_two)

7.5 7.905694150420948 62.5 235.82302686548658


### Conclusion
While the MAE (7.5) to RMSE (7.9056941504209481) ratio was about 1:1 for the first list of errors, the MAE (62.5) to RMSE (235.82302686548658) ratio was closer to 1:4 for the second list of errors. In general, we should expect that the MAE value be much less than the RMSE value. The only difference between the 2 sets of errors is the extreme 1000 value in errors_two instead of 10. When we're working with larger data sets, we can't inspect each value to understand if there's one or some outliers or if all of the errors are systematically higher. Looking at the ratio of MAE to RMSE can help us understand if there are large but infrequent errors. 
Details can be read [here](https://medium.com/human-in-a-machine-world/mae-and-rmse-which-metric-is-better-e60ac3bde13d#.lyc8od1ix) in this post.